# **Especialização em Ciência de Dados - INF/UFRGS E SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---



<br>

## **Aula 07** - **Tópico: Introdução ao Aprendizado Ensemble**

<br>

No que diz respeito a tomada de decisão em tarefas preditivas, **"ouvir a opinião" de um conjunto de árvores de decisão** pode trazer alguma vantagem em termos de desempenho preditivo?

***Vamos investigar essa possibilidade?***

*Observação: O desenvolvimento deste notebook foi baseado no livro de Aurélien Géron, Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow (2nd Edition)*

----


###Carregando e dividindo os dados


Neste notebook, vamos utilizar novamente o dataset [Breast Cancer Wisconsin](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset), carregando-o através das funções do scikit-learn.

In [ ]:
import pandas as pd             # biblioteca para análise de dados 
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
from sklearn.datasets import load_breast_cancer ## conjunto de dados a ser analisado
sns.set()

data = load_breast_cancer() ## carrega os dados de breast cancer
X = data.data  # matriz contendo os atributos
y = data.target  # vetor contendo a classe (0 para maligno e 1 para benigno) de cada instância
feature_names = data.feature_names  # nome de cada atributo
target_names = data.target_names  # nome de cada classe

## Relembrando as características do dataset
print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

Iniciamos fazendo uma divisão dos dados em treino e teste na proporção 80%/20%. Fazemos de forma estratificada, isto é, mantendo a proporção original das classes ('stratify=y').

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 

## Fazemos a divisão com 2-way holdout, de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

O método ShuffleSplit facilita a divisão dos dados em um número pré-determinado de conjuntos de treino e teste, cada par independente entre si. Para mais informações sobre o método, veja [este link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html).

In [ ]:
from sklearn.model_selection import ShuffleSplit 

n_trees = 501
n_instances = 100

mini_sets = []

# 501 árvores a partir de um conjunto limitado de dados

# amostrando 501 conjunto de dados com 100 instancias aleatorias (a partir dos 80% p/ treino)
## cria diversos conjuntos de treino/teste indepenedentes
## deixa 'n_instances' instâncias para treinamento
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
# 100 instâncias para treino

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

###Treinando múltiplas árvores de decisão

A seguir, vamos realizar a otimização de hiperparâmetros de uma árvore de decisão utilizando a função GridSearchCV. Para agilizar o processo a ser executado em aula, vamos adotar duas simplificações: i) iremos testar um número limitado de combinações de valores para os hiperparâmetros, ii) vamos estimar os melhores valores de hiperparâmetros com base nos dados X_train e depois vamos 'copiar' estes valores para 501 novas árvores a serem treinadas nos dados amostrados. Isto garante que todas as árvores utilizarão a melhor configuração de hiperparâmetros encontrada, mas cada árvore será treinada com base em uma amostra específica dos dados (1 dentre as 501 geradas).

In [ ]:
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## otimiza os hiperparâmetros do algoritmo de árvores de decisão usando
## função GridSearchCV do scikit-learn e X_train.
params = {'max_depth': list(range(2, 6)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

## constrói um novo 'estimador' do scikit-learn com os mesmos hiperparâmetros
## mas sem estar ajustados aos dados (ou seja, não copia os dados, apenas hiperparâmetros)
setTrees = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)] # 501 cópias


In [ ]:
## faz o treinamento de cada uma das 'n_trees' árvores
accuracy_scores = []
proba_all = []
for tree, (X_mini_train, y_mini_train) in zip(setTrees, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    proba_all.append(tree.predict_proba(X_test)[:,0])
    accuracy_scores.append(accuracy_score(y_test, y_pred))

Como foi a variação de desempenho entre as árvores? Podemos analisar a acurácia média e a distribuição das acurácias entre as 501 árvores. A mesma análise poderia ser feita para outras medidas de desempenho.

In [ ]:
np.mean(accuracy_scores)

In [ ]:
ax= sns.boxplot(y=accuracy_scores)
ax.set_ylabel("Acurácia")

Também é interessante observar como estas árvores variam entre as predições para os dados de teste (X_test, que são os mesmos para todas as árvores). Vamos criar um heatmap (mapa de calor), usando a probabilidade para a classe 0 (que representa tumor maligno, neste notebook).

In [ ]:
print(len(proba_all))
print(len(proba_all[1]))

## transforma lista em array, para plotar heatmap
proba_all_array = np.asarray(proba_all)
print(proba_all_array.T.shape) ## obtém o tamanho da transposta do array/matriz

A seguir vamos usar um heatmap (mapa de calor) para visualizar as diferenças nas predições entre as várias árvores. As linhas representam as instâncias de teste e as colunas as árvores treinadas.

In [ ]:
## Visualizando as probabilidades para a classe 0.
## Quanto mais próximo de 1, mais provável de ser da classe "Maligno"
fig, ax = plt.subplots(figsize=(16, 16))
ax = sns.heatmap(proba_all_array.T,vmin=0, vmax=1,cmap="YlGnBu")
ax.set(xticklabels=[],yticks=list(range(0,114,2)))
plt.xlabel('Trees')
plt.ylabel('Test instances')
plt.show()

Analise como as árvores possuem diferenças entre si para as instâncias analisadas. Em alguns casos, a predição possui uma clara concordância entre todas as árvores. Em outros, algumas árvores discordam em relação à classe de uma dada instância. Essa diversidade de opiniões pode tornar a tomada de decisão melhor e mais robusta, se houver uma forma apropriada de combinar as opiniões.

###Agregando as predições de múltiplos classificadores

Vamos avaliar o desempenho ao se agregar a decisão destas árvores. Vamos assumir que a saída da tomada de decisão conjunta será com base na **classe mais votada** entre todas as árvores.

In [ ]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

## agrega a classe predita por todas as árvores em um array de 2D
for tree_index, tree in enumerate(setTrees):
    Y_pred[tree_index] = tree.predict(X_test)

## observando a saída gerada para as duas primeiras instâncias de teste 
print(Y_pred[:2,:])

In [ ]:
from scipy.stats import mode

## determina a saída a partir da moda (valor mais frequente)
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

## observa a classe predita pela abordagem de votação
print(y_pred_majority_votes)

In [ ]:
## número de votos que cada classe recebeu.
## é possível perceber a correlação com a análise do heatmap
print(n_votes)

In [ ]:
## avalia o desempenho desta classificação a partir de uma votação majoritária
acc_voting = accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))
print(acc_voting)

Podemos comparar o desempenho do método de votação (que agrega múltiplas saídas) com o desempenho entre as 501 árvores treinadas e avaliadas. A agregação de modelos pode trazer benefícios nas tarefas de análise preditiva, e existe toda uma teoria por traz desta estratégia denominada **wisdom of crowds** (**sabedoria das multidões**).

O exercício desenvolvido neste notebook é praticamente uma implementação "manual" do algoritmo de Florestas Aleatórias, com algumas diferenças propostas pelo algoritmo que iremos estudar a seguir.